In [1]:
import csv

# read flash.dat to a list of lists
datContent = [i.strip().split() for i in open("./densZ0002680.dat").readlines()]

In [17]:
len(datContent)

9702

In [11]:
datContent[100]

['24.000000', '-72.000000', '-120.000000', '0']

5.770969660916121